Authored by Andre Otte, July 2021.

This script is to download and clip all GEDI data that intersects with an area of interest. The script saves GeoJSON files to the file system for each GEDI .h5 download from EarthData. This was created to address the issue of GEDI files containing large amounts of unnecessary data when only looking at a specific area of interest. 

All [level 1B, 2A, and 2B](https://gedi.umd.edu/data/products/) files will be downloaded and all attributes converted to GeoJSON.

This script works in the Colab environment, but honestly, it may be easier to get run locally assuming some familiarity with running Python from the command line. Because download and processing can take up to 10-15 minutes per file, runtime disconnect issues may occur if the browser is left unattended. Additionally, all the processed files will need to be saved locally in order to prevent losing them when running the script a second time.

Potantial areas of improvement:
1. The clipping and processing code was taken from [GEDI Subsetter](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse), a command line tool for clipping a directory of GEDI h5 files and outputting to GeoJSON. If offers more customizability than this notebook. More customizability could be  built into this script.
2. Some of the logic in the GEDI Susetter function could be abstracted out into smaller functions or added to a python package.

TODO - get all the layers instead of just the defaults.

In [1]:
!pip install pyGEDI
!pip install geopandas
!pip install rtree
from pyGEDI import *
from datetime import date
from shapely.geometry import Polygon
import json
import geopandas
import os
import h5py
import pandas as pd
import sys
import numpy as np

  Created wheel for pyGEDI: filename=pyGEDI-0.2.5-cp37-none-any.whl size=4146 sha256=1891e3bbe699c147079b6afe54644960e77344e8beaf8ed157a09b0cec0fc877
  Stored in directory: /root/.cache/pip/wheels/82/ac/2b/90d7bedb2f8a4a4dafcd66cd8f3efa9939c268316256dc11fc
Successfully built pyGEDI
     |████████████████████████████████| 1.0MB 9.4MB/s 
     |████████████████████████████████| 6.6MB 28.0MB/s 
     |████████████████████████████████| 15.4MB 200kB/s 
     |████████████████████████████████| 1.0MB 8.8MB/s 


The following functions are taken from [pyGEDI](https://github.com/EduinHSERNA/pyGEDI/blob/master/pyGEDI/fuctions.py) and modified slightly to us to download one .h5 file at a time, allowing us to clip and process each file before downloading the next one.

In [2]:
def gediDownload(url, outdir, fileName, session):
  """Download the GEDI file from EarthData and save it to a directory named GEDI Product/data collection day
  :param url: The EarthData download link for the .h5 file
  :param outdir: The root directory for the .h5 files
  :param session: The EarthData session
  """
  print(f"    Begin {fileName} download from EarthData.")
  try:
    os.makedirs(outdir)
  except OSError:
    print (f"    WANRING - Creation of the subdirectory {outdir} failed or already exists")
  else:
    print (f"    Created the subdirectory {outdir}")  

  path = outdir + fileName + ".h5"
  
  with open(path, 'wb') as f:
    response = session.get(url, stream=True)
    total = response.headers.get('content-length')
    if total is None:
      f.write(response.content)
    else:
      downloaded = 0
      total = int(total)
      for data in response.iter_content(chunk_size=max(int(total/1000), 1024*1024)):
        downloaded += len(data)
        f.write(data)
        done = int(100*downloaded/total)
        gb=float(total/1073741824)

        sys.stdout.write('\r' + '   ' +url[url.rfind(':')+52:]+' | '+str(gb)[:5]+'GB | '+ str(100*downloaded/total)+ '% [{}{}]'.format('█' * done, '.' * (100 -done)))
        sys.stdout.flush()
  sys.stdout.write('\n')
  print(f"    {fileName} download complete.")


Earthdata has a tool called [GEDI Finder](https://lpdaacsvc.cr.usgs.gov/services/gedifinder) that returns a list of files for a given AOI. This function will call the Gedi Finder Web Service and return the list of files to download.

In [3]:
def getGediDownloadLinks(product, version, bbox):
  """Get a list of download links that intersect an AOI from the GEDI Finder web service.
  :param product: The GEDI product. Options - 1B, 2A, or 2B
  :param version: The GEDI production version. Option - 001
  :param bbox: An area of interest as an array containing the upper left lat, upper left long, lower right lat and lower right long coordinates - 
   [ul_lat,ul_lon,lr_lat,lr_lon]
  """
  bboxStr = bbox[0] + ',' + bbox[1] + ',' +  bbox[2] + ',' + bbox[3]
  url='https://lpdaacsvc.cr.usgs.gov/services/gedifinder?product='+product+'&version='+str(version)+'&bbox='+bboxStr+'&output=json'
  
  print(f"{product} downloads: {url}")

  content=requests.get(url)
  listh5=content.json().get('data')
  return listh5
  

This function was created from the EarthData [GEDI Subsetter](https://lpdaac.usgs.gov/news/release-gedi-subsetter-data-prep-script/) command line tool. It takes in an area of interest and a file path and returns a clipped GeoDataFrame. There are left over, unused chunks of code in here. I did not remove partially due to laziness, but also to leave the option open for modification in the future.

In [4]:
def gediSubsetter(aoi, inFilePath, layers = None):
  """Convert a GEDI h5 file to a GeoDataFrame
  :param aoi: a string list representing the Area of Interest.
  :param inFilePath: The file path of the GEDI h5 file.
  :param layers (optional): A list of layers that will be extracted out of the h5 file.
  """  
  ROI = [float(aoi[0]), float(aoi[1]), float(aoi[2]), float(aoi[3])]

  try:
      ROI = Polygon([(ROI[1], ROI[0]), (ROI[3], ROI[0]), (ROI[3], ROI[2]), (ROI[1], ROI[2])]) 
      ROI.crs = 'EPSG:4326'
  except:
      print('    ERROR - unable to read input bounding box coordinates, the required format is: ul_lat,ul_lon,lr_lat,lr_lon')
      sys.exit(2)

  # Keep the exact input geometry for the final clip to ROI
  finalClip = geopandas.GeoDataFrame([1], geometry=[ROI], crs='EPSG:4326')    
  
  # Find input directory
  try:
      os.chdir(inFilePath)
  except FileNotFoundError:
      print('    ERROR - input directory (--dir) provided does not exist or was not found')
      sys.exit(2)

  beamSubset = ['BEAM0000', 'BEAM0001', 'BEAM0010', 'BEAM0011', 'BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']
  layerSubset = None
      
  # -------------------------------------SET UP WORKSPACE------------------------------------------ #
  # Create list of GEDI HDF-EOS5 files in the directory
  gediFiles = [o for o in os.listdir() if o.endswith('.h5') and 'GEDI' in o]

  # --------------------DEFINE PRESET BAND/LAYER SUBSETS ------------------------------------------ #
  # Default layers to be subset and exported, see README for information on how to add additional layers
  l1bSubset = [ '/geolocation/latitude_bin0', '/geolocation/longitude_bin0', '/channel', '/shot_number',
              '/rxwaveform','/rx_sample_count', '/stale_return_flag', '/tx_sample_count', '/txwaveform',
              '/geolocation/degrade', '/geolocation/delta_time', '/geolocation/digital_elevation_model',
              '/geolocation/solar_elevation',  '/geolocation/local_beam_elevation',  '/noise_mean_corrected',
              '/geolocation/elevation_bin0', '/geolocation/elevation_lastbin', '/geolocation/surface_type', '/geolocation/digital_elevation_model_srtm']
  l2aSubset = ['/lat_lowestmode', '/lon_lowestmode', '/channel', '/shot_number', '/degrade_flag', '/delta_time', 
              '/digital_elevation_model', '/elev_lowestmode', '/quality_flag', '/rh', '/sensitivity', '/digital_elevation_model_srtm', 
              '/elevation_bias_flag', '/surface_flag',  '/num_detectedmodes',  '/selected_algorithm',  '/solar_elevation']
  l2bSubset = ['/geolocation/lat_lowestmode', '/geolocation/lon_lowestmode', '/channel', '/geolocation/shot_number',
              '/cover', '/cover_z', '/fhd_normal', '/pai', '/pai_z',  '/rhov',  '/rhog',
              '/pavd_z', '/l2a_quality_flag', '/l2b_quality_flag', '/rh100', '/sensitivity',  
              '/stale_return_flag', '/surface_flag', '/geolocation/degrade_flag',  '/geolocation/solar_elevation',
              '/geolocation/delta_time', '/geolocation/digital_elevation_model', '/geolocation/elev_lowestmode']

  # -------------------IMPORT GEDI FILES AS GEODATAFRAMES AND CLIP TO ROI-------------------------- #   
  # Loop through each GEDI file and export as a point geojson
  l = 0
  for g in gediFiles:
      l += 1
      print(f"    Processing file: {g} ({l}/{len(gediFiles)})")
      gedi = h5py.File(g, 'r')      # Open file
      gediName = g.split('.h5')[0]  # Keep original filename
      gedi_objs = []            
      gedi.visit(gedi_objs.append)  # Retrieve list of datasets  

      # Search for relevant SDS inside data file
      gediSDS = [str(o) for o in gedi_objs if isinstance(gedi[o], h5py.Dataset)] 
      
      # Define subset of layers based on product. If layers param comes in as null, use the dedault layers defined above. 
      if layers == None:
        if 'GEDI01_B' in g:
            sdsSubset = l1bSubset
        elif 'GEDI02_A' in g:
            sdsSubset = l2aSubset 
        else:
            sdsSubset = l2bSubset
      else:
        sdsSubset = layers
  
      # Append additional datasets if provided
      if layerSubset is not None:
          [sdsSubset.append(y) for y in layerSubset]

      # Subset to the selected datasets
      gediSDS = [c for c in gediSDS if any(c.endswith(d) for d in sdsSubset)]
      
      # Get unique list of beams and subset to user-defined subset or default (all beams)
      beams = []
      for h in gediSDS:
          beam = h.split('/', 1)[0]
          if beam not in beams and beam in beamSubset:
              beams.append(beam)

      gediDF = pd.DataFrame()  # Create empty dataframe to store GEDI datasets    
      del beam, gedi_objs, h
      
      # Loop through each beam and create a geodataframe with lat/lon for each shot, then clip to ROI
      for b in beams:
          beamSDS = [s for s in gediSDS if b in s]
          
          # Search for latitude, longitude, and shot number SDS
          lat = [l for l in beamSDS if sdsSubset[0] in l][0]  
          lon = [l for l in beamSDS if sdsSubset[1] in l][0]
          shot = f'{b}/shot_number'          
          
          # Open latitude, longitude, and shot number SDS
          shots = gedi[shot][()]
          lats = gedi[lat][()]
          lons = gedi[lon][()]
          
          # Append BEAM, shot number, latitude, longitude and an index to the GEDI dataframe
          geoDF = pd.DataFrame({'BEAM': len(shots) * [b], shot.split('/', 1)[-1].replace('/', '_'): shots,
                                'Latitude':lats, 'Longitude':lons, 'index': np.arange(0, len(shots), 1)})
          
          # Convert lat/lon coordinates to shapely points and append to geodataframe
          geoDF = geopandas.GeoDataFrame(geoDF, geometry=geopandas.points_from_xy(geoDF.Longitude, geoDF.Latitude))
          
          # Clip to only include points within the user-defined bounding box
          geoDF = geoDF[geoDF['geometry'].within(ROI.envelope)]    
          gediDF = gediDF.append(geoDF)
          del geoDF
      
      # Convert to geodataframe and add crs
      gediDF = geopandas.GeoDataFrame(gediDF)
      gediDF.crs = 'EPSG:4326'
      
      if gediDF.shape[0] == 0:
          print(f"    WANRING - No intersecting shots were found between {g} and the region of interest submitted.")
          continue
      del lats, lons, shots
      
  # --------------------------------OPEN SDS AND APPEND TO GEODATAFRAME---------------------------- #
      beamsDF = pd.DataFrame()  # Create dataframe to store SDS
      j = 0
      
      # Loop through each beam and extract subset of defined SDS
      for b in beams:
          beamDF = pd.DataFrame()
          beamSDS = [s for s in gediSDS if b in s and not any(s.endswith(d) for d in sdsSubset[0:3])]
          shot = f'{b}/shot_number'
          
          try:
              # set up indexes in order to retrieve SDS data only within the clipped subset from above
              mindex = min(gediDF[gediDF['BEAM'] == b]['index'])
              maxdex = max(gediDF[gediDF['BEAM'] == b]['index']) + 1
              shots = gedi[shot][mindex:maxdex]
          except ValueError:
              print(f"    WARNING - No intersecting shots found for {b}")
              continue
          # Loop through and extract each SDS subset and add to DF
          for s in beamSDS:
              j += 1
              sName = s.split('/', 1)[-1].replace('/', '_')

              # Datasets with consistent structure as shots
              if gedi[s].shape == gedi[shot].shape:
                  beamDF[sName] = gedi[s][mindex:maxdex]  # Subset by index
              
              # Datasets with a length of one 
              elif len(gedi[s][()]) == 1:
                  beamDF[sName] = [gedi[s][()][0]] * len(shots) # create array of same single value
              
              # Multidimensional datasets
              elif len(gedi[s].shape) == 2 and 'surface_type' not in s: 
                  allData = gedi[s][()][mindex:maxdex]
                  
                  # For each additional dimension, create a new output column to store those data
                  for i in range(gedi[s].shape[1]):
                      step = []
                      for a in allData:
                          step.append(a[i])
                      beamDF[f"{sName}_{i}"] = step
              
              # Waveforms
              elif s.endswith('waveform') or s.endswith('pgap_theta_z'):
                  waveform = []
                  
                  if s.endswith('waveform'):
                      # Use sample_count and sample_start_index to identify the location of each waveform
                      start = gedi[f'{b}/{s.split("/")[-1][:2]}_sample_start_index'][mindex:maxdex]
                      count = gedi[f'{b}/{s.split("/")[-1][:2]}_sample_count'][mindex:maxdex]
                  
                  # for pgap_theta_z, use rx sample start index and count to subset
                  else:
                      # Use sample_count and sample_start_index to identify the location of each waveform
                      start = gedi[f'{b}/rx_sample_start_index'][mindex:maxdex]
                      count = gedi[f'{b}/rx_sample_count'][mindex:maxdex]
                  wave = gedi[s][()]
                  
                  # in the dataframe, each waveform will be stored as a list of values
                  for k in range(len(start)):
                      singleWF = wave[int(start[k] - 1): int(start[k] - 1 + count[k])]
                      waveform.append(','.join([str(q) for q in singleWF]))
                  beamDF[sName] = waveform
              
              # Surface type 
              elif s.endswith('surface_type'):
                  surfaces = ['land', 'ocean', 'sea_ice', 'land_ice', 'inland_water']
                  allData = gedi[s][()]
                  for i in range(gedi[s].shape[0]):
                      beamDF[f'{surfaces[i]}'] = allData[i][mindex:maxdex]
                  del allData
              else:
                  print(f"    SDS: {s} not found")
          
          beamsDF = beamsDF.append(beamDF)
      del beamDF, beamSDS, beams, gedi, gediSDS, shots, sdsSubset
      
      # Combine geolocation dataframe with SDS layer dataframe
      outDF = pd.merge(gediDF, beamsDF, left_on='shot_number', right_on=[sn for sn in beamsDF.columns if sn.endswith('shot_number')][0])
      outDF.index = outDF['index']
      del gediDF, beamsDF   
      
      # Subset the output DF to the actual boundary of the input ROI
      outDF = geopandas.overlay(outDF, finalClip)
      del outDF[0] 

      return outDF

Set up an EarthData session and the area of interest bounding box. This is the only block that contains hard-coded values that can change.



In [5]:
username=""
password=""
session=sessionNASA(username,password)
rootDirectory = "data"
isColabEnvironment = True

#The list of GEDI products
product_1B='GEDI01_B'
product_2A='GEDI02_A'
product_2B='GEDI02_B'

#The GEDI product version
version='001'

#The Area of Interest
ul_lat= '-13.76913'
ul_lon= '-44.0654'
lr_lat= '-13.67646'
lr_lon= '-44.17246'

bbox=[ul_lat, ul_lon, lr_lat, lr_lon]

Get the download links for the GEDI files that intersect with the area of interest. Following the link will prompt a sign in to EarthData, after which the download will begin.

In [6]:
downloadList2B = getGediDownloadLinks(product_2B,version,bbox)
downloadList2A = getGediDownloadLinks(product_2A,version,bbox)
downloadList1B = getGediDownloadLinks(product_1B,version,bbox)

downloadList = downloadList2B + downloadList2A + downloadList1B
downloadList

GEDI02_B downloads: https://lpdaacsvc.cr.usgs.gov/services/gedifinder?product=GEDI02_B&version=001&bbox=-13.76913,-44.0654,-13.67646,-44.17246&output=json
GEDI02_A downloads: https://lpdaacsvc.cr.usgs.gov/services/gedifinder?product=GEDI02_A&version=001&bbox=-13.76913,-44.0654,-13.67646,-44.17246&output=json
GEDI01_B downloads: https://lpdaacsvc.cr.usgs.gov/services/gedifinder?product=GEDI01_B&version=001&bbox=-13.76913,-44.0654,-13.67646,-44.17246&output=json


['https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2020.07.20/GEDI02_B_2020202065047_O09083_T05345_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2020.04.21/GEDI02_B_2020112181526_O07695_T03922_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2020.04.17/GEDI02_B_2020108194843_O07634_T02652_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2020.01.10/GEDI02_B_2020010104413_O06109_T02652_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2019.11.10/GEDI02_B_2019314105245_O05163_T01076_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2019.10.14/GEDI02_B_2019287090415_O04743_T02491_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2019.08.30/GEDI02_B_2019242025913_O04041_T01068_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2019.07.17/GEDI02_B_2019198085218_O03362_T03922_02_001_01.h5',
 'https://e4ftl01.cr.usgs.gov/GEDI/GEDI02_B.001/2019.05.27/GEDI02_B_2019147050950_O02568_T05345_02_001_01.h5',
 

The following logic flows as follows:

Foreach file we need to download -
1. Get the path the file will be downloaded to.
2. Download the file using the GEDI Finder web service.
3. Determine the list of laters we need to extract from the h5 file.
4. Using the gediSubsetter function, clip the h5 file, and convert the data to geojson.
5. Delete the raw h5 file. 

In [9]:
count = 1
# origList = downloadList
# downloadList = [downloadList[0], downloadList[1], downloadList[11], downloadList[12], downloadList[21], downloadList[22]]
# downloadList = [downloadList[15],downloadList[15]]
# for url in downloadList:
for url in [downloadList[5],downloadList[6],downloadList[15],downloadList[16],downloadList[25],downloadList[26]]:
  #In the colab environment, the folders end up being nested. We need to cd into the root directory after each iteration.
  if(isColabEnvironment):
    os.chdir('/content/')
  
  #Get the name of the file we just downloaded and saved.
  fileNameh5 = re.search("GEDI\d{2}_\D_.*", url).group(0).replace(".h5", "") # regex matches GEDI{01 or 02}_{A or B}_.*
  day = re.search("\d{4}\.\d{2}\.\d{2}", url).group(0) # regex matches date formatted 'yyyy.mm.dd'
  outdir = rootDirectory + os.sep + re.search("GEDI\d{2}_\D\.\d{3}", url).group(0) + os.sep + day + os.sep #regex matches GEDI{01 or 02}_{A or B}.001
  product = re.search("GEDI\d{2}_\D", url).group(0)
  filePathH5 = outdir + fileNameh5 + ".h5"

  print(f"BEGIN DOWNLOAD AND PROCESSING {fileNameh5}. FILE {count} OF {str(len(downloadList))}.")

  #If the file exists in the filesystem, skip the download.  
  if not os.path.isfile(filePathH5):
    gediDownload(url, outdir, fileNameh5, session)
  else:
    print(f"    File {fileNameh5} exists in file system. Skipping download.")
  
  #Set up the layers to extract from GEDI file
  h5_2B=getH5(filePathH5)

  #This returns a dictionary with {file name: [comma separated layers]} file name as the key and list of layers as the value
  layers = getLayer('',[h5_2B])[filePathH5] 
  
  #The subsetter function needs the coordinate layers to be the first two layers in the list.
  if product == 'GEDI02_B':
    layers.insert(0, layers.pop(layers.index('geolocation/lat_lowestmode')))
    layers.insert(1, layers.pop(layers.index('geolocation/lon_lowestmode')))
  elif product == 'GEDI01_B':
    layers.insert(0, layers.pop(layers.index('geolocation/latitude_bin0')))
    layers.insert(1, layers.pop(layers.index('geolocation/longitude_bin0')))
  else: #product = 'GEDI02_A'
    layers.insert(0, layers.pop(layers.index('lat_lowestmode')))
    layers.insert(1, layers.pop(layers.index('lon_lowestmode')))

  # geodataframe = gediSubsetter(bbox, outdir, layers) #Call to the subsetter with defined layers.
  geodataframe = gediSubsetter(bbox, outdir) #Call to the subsetter which will use the default laters.

  #Add columns to the dataframe
  geodataframe['gedi_product'] = product
  geodataframe['gefi_file_name'] = fileNameh5
  geodataframe['data_collected_date'] = day
  geodataframe['accessed_date'] = str(date.today())

  print("    Sample data from geodataframe:")
  print(geodataframe.head())
  
  #Convert to GeoJSON
  print("    Converting geodataframe to json.")
  geodataframe.to_file(fileNameh5 + '.json', driver="GeoJSON")
 
  print(f"FINISHED PROCESSING {fileNameh5}.")
  print("------------------------------------")
  count += 1
  
  #In the colab environment, the folders end up being nested. We need to cd into the root directory after each iteration.
  if(isColabEnvironment):
    os.chdir('/content/')
  #Remove the raw .h5 file from the file system. 
  try:
    os.remove(filePathH5)
  except FileNotFoundError:
      print(f"    WARNING - h5 file failed to be removed.")
      continue

BEGIN DOWNLOAD AND PROCESSING GEDI02_B_2019287090415_O04743_T02491_02_001_01. FILE 1 OF 30.
    Begin GEDI02_B_2019287090415_O04743_T02491_02_001_01 download from EarthData.
    Created the subdirectory data/GEDI02_B.001/2019.10.14/
   GEDI02_B_2019287090415_O04743_T02491_02_001_01.h5 | 1.874GB | 100.0% [████████████████████████████████████████████████████████████████████████████████████████████████████]
    GEDI02_B_2019287090415_O04743_T02491_02_001_01 download complete.
    Processing file: GEDI02_B_2019287090415_O04743_T02491_02_001_01.h5 (1/1)
    Sample data from geodataframe:
       BEAM        shot_number  ...  data_collected_date  accessed_date
0  BEAM0000  47430016100109663  ...           2019.10.14     2021-07-17
1  BEAM0000  47430016300109664  ...           2019.10.14     2021-07-17
2  BEAM0000  47430016500109665  ...           2019.10.14     2021-07-17
3  BEAM0000  47430016700109666  ...           2019.10.14     2021-07-17
4  BEAM0000  47430016900109667  ...           2019

Combine the JSONs from each product into a single file.

All files share the same 'type' and 'crs' attributes. The 'features' attribute holds the data.
Example structure:



```
{
    "type": "FeatureCollection",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
        }
    },
    "features": [{
            "type": "Feature",
            "properties": {
                "BEAM": "0",
                "shot_number_x": 90830000500534134
            },
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -44.17234047170119,
                    -13.743702059659597
                ]
            }
      }]
}
```



In [8]:
features = []
firstPass = True
headerType = ""
headerCRS = ""

products = [product_1B + '.' + version, product_2A + '.' + version, product_2B + '.' + version]

for p in products:
  features = []
  print (f"combining {p} json files from /content/data/{p}")
  for root, dirs, files in os.walk(f"/content/data/{p}"):
    for file in files:
      if file.endswith(".json"):
        print(" file: " + os.path.join(root, file))
        with open(os.path.join(root, file)) as f:
          jsonFile = json.load(f)
          headerType = jsonFile["type"]
          headerCRS = jsonFile["crs"]
          dataOne=jsonFile["features"] #this is a list
          features = features + dataOne
          firstPass = False
  
  print(f"after getting the data. product: {p}. ")      
  jsonDict = {
    "type": headerType,
    "crs": headerCRS,
    "features": features
  }

  #If the features attribute is empty, there were no json files found.
  if jsonDict["features"] == []: 
    print(" no data found")
    continue;

  combinedJsonFile = p + ".json"
  with open(combinedJsonFile, 'w') as json_file:
    json.dump(jsonDict, json_file)

  print(" Complete")

combining GEDI01_B.001 json files from /content/data/GEDI01_B.001
 file: /content/data/GEDI01_B.001/2019.10.14/GEDI01_B_2019287090415_O04743_T02491_02_003_01.json
 file: /content/data/GEDI01_B.001/2019.08.30/GEDI01_B_2019242025913_O04041_T01068_02_003_01.json
after getting the data. product: GEDI01_B.001. 
 Complete
combining GEDI02_A.001 json files from /content/data/GEDI02_A.001
 file: /content/data/GEDI02_A.001/2019.10.14/GEDI02_A_2019287090415_O04743_T02491_02_001_01.json
 file: /content/data/GEDI02_A.001/2019.08.30/GEDI02_A_2019242025913_O04041_T01068_02_001_01.json
after getting the data. product: GEDI02_A.001. 
 Complete
combining GEDI02_B.001 json files from /content/data/GEDI02_B.001
 file: /content/data/GEDI02_B.001/2019.10.14/GEDI02_B_2019287090415_O04743_T02491_02_001_01.json
 file: /content/data/GEDI02_B.001/2019.08.30/GEDI02_B_2019242025913_O04041_T01068_02_001_01.json
after getting the data. product: GEDI02_B.001. 
 Complete
